# Import

In [1]:
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

In [2]:
from dotenv import load_dotenv, find_dotenv
import sys
import os

load_dotenv(find_dotenv())


True

# Graph


In [66]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState 
from langchain_core.messages import AIMessage
import time
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate


llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")

workflow = StateGraph(MessagesState)
PROMPT_INFOR_AGENT  = """
"You are a get user information agent.
USER INFORMATION:
- Name: Phan Van Thien
- Age: 22
- Occupation: AI developer
- Location: Vietnam
- Interests: AI, technology, programming
- Skills: Python, machine learning, deep learning
- Education: Bachelor's degree in Computer Science
- Experience: 2 years in AI development
- Languages: Vietnamese (native), English (fluent)
INSTRUCTIONS:
- Assist ONLY with user information-related tasks.
- Respond ONLY with the final result of your work, do NOT include ANY other text."
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", PROMPT_INFOR_AGENT),
        ("user", "{messages}"),
    ]
)
chain = prompt | llm 
def get_in4 (state):  
    messages = state.get("messages", [])
    messages = "\n".join([f"{message.role}{message.content}" for message in messages])
    return chain.invoke(messages)
workflow.add_node("chat",
                lambda state: {"messages": [chain.invoke(state["messages"])]},
                )
workflow.add_edge(START, "chat")
workflow.add_edge("chat", END)
graph = workflow.compile(name = "get_user_information_agent")

In [ ]:
graph_output = graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is user's name?"
            }
        ]
    }
)

================================ Human Message =================================

What is user's name?
================================== Ai Message ==================================

Phan Van Thien


In [4]:
from langchain_core.tools import tool
# Create specialized agents
@tool
def add(a: float, b: float) -> float:
    '''Add two numbers.'''
    return a + b
@tool
def subtract(a: float, b: float) -> float:
    '''Subtract two numbers.'''
    return a - b
@tool
def multiply(a: float, b: float) -> float:
    '''Multiply two numbers.'''
    return a * b
@tool
def divide(a: float, b: float) -> float:
    '''Divide two numbers.'''
    if b == 0:
        raise ValueError("Cannot divide by zero.")
    return a / b
@tool
def exponentiate(a: float, b: float) -> float:
    '''Raise a number to the power of another.'''
    return a ** b

PROMPT_MATH_AGENT = """
"You are a math agent.
INSTRUCTIONS:
- Assist ONLY with math-related tasks.
- For complex expressions involving multiple operations, break them down into smaller steps.
- Execute one operation at a time using the available tools.
- Clearly state the intermediate results if needed.
- After you're done with all calculations, respond to the supervisor directly.
- Respond ONLY with the final result of your work, do NOT include ANY other text."
"""
math_agent = create_react_agent(
    model= llm,
    tools=[add, subtract, multiply, divide, exponentiate],
    prompt= PROMPT_MATH_AGENT,
    name="math_expert",
)
math_output = math_agent.invoke({
    "messages": {
        "role": "user",
        "content": "What is 2 + 100 - 10 * 10?"
    }
    })

In [5]:
from langchain_core.messages import convert_to_messages
def pretty_print_message ( message , indent = False ) :
    pretty_message = message . pretty_repr ( html = True )
    if not indent :
        print ( pretty_message )
        return
    indented = "\n". join ("\t" + c for c in pretty_message . split ("\n") )
    print ( indented )
def pretty_print_messages ( messages , indent = False ) :
    for message in messages :
        pretty_print_message ( message , indent )

In [8]:
from langchain.tools.google_serper import GoogleSerperResults

search = GoogleSerperResults()

research_agent = create_react_agent(
    model=llm,
    tools=[search],
    prompt="""
        You are a research assistant. 
        INSTRUCTIONS:
        - Assist ONLY with research-related tasks.
        - For complex queries, break them down into smaller steps.
        - After you're done with your tasks, respond to the supervisor directly
        - Respond ONLY with the results of your work, do NOT include ANY other text.
    """,
    name="research_assistant",
)
research_agent_output = research_agent.invoke({
    "messages": {
        "role": "user",
        "content": "What is the population of France?"
    }
})

In [9]:
pretty_print_messages(research_agent_output["messages"], indent=True)

	================================ Human Message =================================
	
	What is the population of France?
	================================== Ai Message ==================================
	Name: research_assistant
	Tool Calls:
	  google_serper_results_json (fe62db60-b0a8-4dc5-bc6a-173bb5b857e2)
	 Call ID: fe62db60-b0a8-4dc5-bc6a-173bb5b857e2
	  Args:
	    query: population of France
	================================= Tool Message =================================
	Name: google_serper_results_json
	
	{'searchParameters': {'q': 'population of France', 'gl': 'us', 'hl': 'en', 'type': 'search', 'num': 10, 'engine': 'google'}, 'answerBox': {'title': 'France / Population', 'answer': '68.29 million (2023)', 'source': 'World Bank', 'sourceLink': 'http://datatopics.worldbank.org/world-development-indicators'}, 'organic': [{'title': 'France Population (2025) - Worldometer', 'link': 'https://www.worldometers.info/world-population/france-population/', 'snippet': "The current populatio

In [67]:
from langgraph_supervisor import create_supervisor

supervisor = create_supervisor(
    model = llm,
    agents = [research_agent, math_agent, graph],
    prompt = (
        "You are a supervisor managing two agents:\n"
        "- a research agent. Only Assign research-related tasks to this agent\n"
        "- a math agent. Only Assign math-related tasks to this agent\n"
        "- a get_user_information_agent. Only Assign user information-related tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Only call agent to complete tasks except planning. Do not do any work yourself."
        "With complex tasks, break them down into smaller steps and solve them step by step \n"
    ),
    add_handoff_back_messages=True,
    output_mode="full_history",
).compile()

In [69]:
output = supervisor.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Find the minimum salary of user's occupation and user's age and user's age to the min salary"
            }
        ]
    }
)
output

{'messages': [HumanMessage(content="Find the minimum salary of user's occupation and user's age and user's age to the min salary", additional_kwargs={}, response_metadata={}, id='eefdf4bc-0f99-41c9-b3e2-e9af50a06ea8'),
  AIMessage(content="I need to get the user's occupation and age first. I will ask the get_user_information_agent to get this information.", additional_kwargs={'function_call': {'name': 'transfer_to_get_user_information_agent', 'arguments': '{}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, name='supervisor', id='run--51114ebd-4b67-4249-8862-7ab6d88ece12-0', tool_calls=[{'name': 'transfer_to_get_user_information_agent', 'args': {}, 'id': '6dbacfbb-9cd6-4da2-b1e0-27816266cdc4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 190, 'output_tokens': 41, 'total_tokens': 231, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='Successfully transferred to get_user_informa